In [ ]:
import os
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories

In [ ]:
%pwd

In [ ]:
%cd ..

In [ ]:
%pwd

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import  load_from_disk
import torch

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainer:
    root_dir: Path 
    data_path: Path
    model_ckpt: str
    num_train_epochs: int = 1
    warmup_steps: int = 500
    per_device_train_batch_size: int = 1
    per_device_eval_batch_size: int = 1
    weight_decay: float = 0.01
    logging_steps: int = 10
    evaluation_strategy: str = 'steps'
    eval_steps: int = 500
    save_steps: float = 1e6
    gradient_accumulation_steps: int = 16


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.root_dir, num_train_epochs=self.num_train_epochs, warmup_steps=self.warmup_steps,
            per_device_train_batch_size=self.per_device_train_batch_size, per_device_eval_batch_size=self.per_device_train_batch_size,
            weight_decay=self.weight_decay, logging_steps=self.logging_steps,
            evaluation_strategy=self.evaluation_strategy, eval_steps=self.eval_steps, save_steps=1e6,
            gradient_accumulation_steps=self.gradient_accumulation_steps
        ) 


        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.root_dir,"tokenizer"))

In [ ]:
class ConfigHandler:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        """
        Initialize the ConfigHandler with the provided configuration and parameters file paths.

        Args:
            config_filepath: Path to the configuration file (default: CONFIG_FILE_PATH)
            params_filepath: Path to the parameters file (default: PARAMS_FILE_PATH)

        Returns:
            None

        """
        self.config = self._load_config(config_filepath)  # Load the configuration from the specified file
        self.params = self._load_params(params_filepath)  # Load the parameters from the specified file

        create_directories([self.config.artifacts_root])  # Create necessary directories based on the configuration

    def _load_config(self, config_filepath):
        """
        Load the configuration from the specified YAML file.

        Args:
            config_filepath: Path to the configuration file

        Returns:
            The loaded configuration as a dictionary

        """
        return read_yaml(config_filepath)  # Read and return the configuration from the YAML file

    def _load_params(self, params_filepath):
        """
        Load the parameters from the specified YAML file.

        Args:
            params_filepath: Path to the parameters file

        Returns:
            The loaded parameters as a dictionary

        """
        return read_yaml(params_filepath)  # Read and return the parameters from the YAML file

    
    
    def get_model_trainer_config(self) -> ModelTrainer:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainer(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [ ]:
try:
    config = ConfigHandler()
    model_trainer = config.get_model_trainer_config()
    model_trainer.train()
except Exception as e:
    raise e